# **AI TECH INSTITUTE** · *Intermediate AI & Data Science*
### Week 01 · Notebook 03 — Data Wrangling & Transformation
**Instructor:** Amir Charkhi  |  **Goal:** Master real-world data manipulation techniques.

> Format: practical scenarios → powerful pandas methods → data ready for analysis.


---
## Real Data is Messy!
Let's load and clean actual messy data - the skills you'll use every day.

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# Set display options for better visibility
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)

print("Ready to wrangle! 🛠️")

Ready to wrangle! 🛠️


## 1. Reading Data from Files

In [3]:
# Create sample CSV data (simulating a file)
csv_data = """order_id,customer_name,product,quantity,price,order_date
1001,Alice Smith,Laptop,1,1200.00,2025-08-15
1002,Bob Jones,Mouse,2,25.50,2025-08-15
1003,Charlie Brown,,1,80.00,2025-08-16
1004,Alice Smith,Monitor,1,,2025-08-16
1005,Diana Prince,Keyboard,3,75.00,2025-08-17
1006,,Webcam,1,120.00,2025-08-17
1007,Bob Jones,Laptop,1,1200,2025-08-18"""

# Save to file and read back
with open('orders.csv', 'w') as f:
    f.write(csv_data)

# Read CSV with proper data types
orders_df = pd.read_csv('orders.csv', parse_dates=['order_date'])
print("Raw data from CSV:")
print(orders_df)
print(f"\nData types:")
print(orders_df.dtypes)
print(f"\nMissing values:")
print(orders_df.isnull().sum())

Raw data from CSV:
   order_id  customer_name   product  quantity   price order_date
0      1001    Alice Smith    Laptop         1  1200.0 2025-08-15
1      1002      Bob Jones     Mouse         2    25.5 2025-08-15
2      1003  Charlie Brown       NaN         1    80.0 2025-08-16
3      1004    Alice Smith   Monitor         1     NaN 2025-08-16
4      1005   Diana Prince  Keyboard         3    75.0 2025-08-17
5      1006            NaN    Webcam         1   120.0 2025-08-17
6      1007      Bob Jones    Laptop         1  1200.0 2025-08-18

Data types:
order_id                  int64
customer_name            object
product                  object
quantity                  int64
price                   float64
order_date       datetime64[ns]
dtype: object

Missing values:
order_id         0
customer_name    1
product          1
quantity         0
price            1
order_date       0
dtype: int64


## 2. Cleaning Missing and Incorrect Data

In [4]:
# Make a copy for cleaning
clean_df = orders_df.copy()

# Handle missing customer names
clean_df['customer_name'].fillna('Unknown Customer', inplace=True)

# Handle missing products (look at other orders from same customer)
clean_df.loc[2, 'product'] = 'Keyboard'  # Reasonable guess based on price

# Handle missing prices (use average for that product)
monitor_avg_price = 350.00  # Domain knowledge
clean_df.loc[3, 'price'] = monitor_avg_price

# Ensure price is float
clean_df['price'] = pd.to_numeric(clean_df['price'], errors='coerce')

# Calculate total
clean_df['total'] = clean_df['quantity'] * clean_df['price']

print("Cleaned data:")
print(clean_df)
print(f"\nRevenue by customer:")
print(clean_df.groupby('customer_name')['total'].sum().sort_values(ascending=False))

Cleaned data:
   order_id     customer_name   product  quantity   price order_date   total
0      1001       Alice Smith    Laptop         1  1200.0 2025-08-15  1200.0
1      1002         Bob Jones     Mouse         2    25.5 2025-08-15    51.0
2      1003     Charlie Brown  Keyboard         1    80.0 2025-08-16    80.0
3      1004       Alice Smith   Monitor         1   350.0 2025-08-16   350.0
4      1005      Diana Prince  Keyboard         3    75.0 2025-08-17   225.0
5      1006  Unknown Customer    Webcam         1   120.0 2025-08-17   120.0
6      1007         Bob Jones    Laptop         1  1200.0 2025-08-18  1200.0

Revenue by customer:
customer_name
Alice Smith         1550.0
Bob Jones           1251.0
Diana Prince         225.0
Unknown Customer     120.0
Charlie Brown         80.0
Name: total, dtype: float64


C:\Users\bduong\AppData\Local\Temp\ipykernel_11112\2590105866.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  clean_df['customer_name'].fillna('Unknown Customer', inplace=True)


In [5]:
orders_df.notnull().mean()

order_id         1.000000
customer_name    0.857143
product          0.857143
quantity         1.000000
price            0.857143
order_date       1.000000
dtype: float64

In [6]:
for col in orders_df.select_dtypes(include='number'):
    print(col)

order_id
quantity
price


**Exercise 1 — Data Quality Check (medium)**  
Create a function that returns a data quality report: % complete, unique counts, and outliers.


In [7]:
# Your turn
def quality_check(df):
    report = pd.DataFrame(index=df.columns)
    
    # % Complete
    report['% Complete'] = df.notnull().mean() * 100
    
    # Unique Counts
    report['Unique Count'] = df.nunique()
    
    # Outliers (using IQR method)
    outlier_counts = {}
    for col in df.select_dtypes(include='number'):
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((df[col] < Q1 - 1.5 * IQR) | (df[col] > Q3 + 1.5 * IQR)).sum()
        outlier_counts[col] = outliers
    report['Outliers'] = pd.Series(outlier_counts)
    
    return report

quality_check(orders_df)
        
        
    




,% Complete,Unique Count,Outliers
order_id,100.000000,7,0.0
customer_name,85.714286,4,NaN
product,85.714286,5,NaN
quantity,100.000000,3,1.0
price,85.714286,5,0.0
order_date,100.000000,4,NaN


<details>
<summary><b>Solution</b></summary>

```python
def data_quality_report(df):
    report = {}
    
    # Completeness
    report['completeness'] = (1 - df.isnull().sum() / len(df)) * 100
    
    # Unique counts
    report['unique_counts'] = df.nunique()
    
    # Outliers for numeric columns (using IQR)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    outliers = {}
    for col in numeric_cols:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers[col] = ((df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)).sum()
    report['outliers'] = outliers
    
    return report

quality_report = data_quality_report(clean_df)
print("Data Quality Report:")
for key, value in quality_report.items():
    print(f"\n{key}:")
    print(value)
```
</details>

## 3. Merging and Joining DataFrames

In [78]:
# Create related dataframes
customers = pd.DataFrame({
    'customer_id': [1, 2, 3, 4],
    'name': ['Alice Smith', 'Bob Jones', 'Charlie Brown', 'Diana Prince'],
    'city': ['Perth', 'Sydney', 'Melbourne', 'Brisbane'],
    'member_since': ['2023-01-15', '2023-06-20', '2024-02-10', '2024-08-01']
})

orders = pd.DataFrame({
    'order_id': [101, 102, 103, 104, 105],
    'customer_id': [1, 2, 1, 3, 1],
    'amount': [150, 250, 100, 300, 175],
    'date': pd.date_range('2025-08-20', periods=5)
})

print("Customers:")
print(customers)
print("\nOrders:")
print(orders)

# Merge dataframes
merged = pd.merge(orders, customers, on='customer_id', how='left')
print("\nMerged data:")
print(merged)

# Different join types
print("\nInner join (only matching):")
inner_join = pd.merge(orders, customers, on='customer_id', how='inner')
print(f"Rows: {len(inner_join)}")

print("\nOuter join (all records):")
outer_join = pd.merge(orders, customers, on='customer_id', how='outer', indicator=True)
print(outer_join)

Customers:
   customer_id           name       city member_since
0            1    Alice Smith      Perth   2023-01-15
1            2      Bob Jones     Sydney   2023-06-20
2            3  Charlie Brown  Melbourne   2024-02-10
3            4   Diana Prince   Brisbane   2024-08-01

Orders:
   order_id  customer_id  amount       date
0       101            1     150 2025-08-20
1       102            2     250 2025-08-21
2       103            1     100 2025-08-22
3       104            3     300 2025-08-23
4       105            1     175 2025-08-24

Merged data:
   order_id  customer_id  amount       date           name       city  \
0       101            1     150 2025-08-20    Alice Smith      Perth   
1       102            2     250 2025-08-21      Bob Jones     Sydney   
2       103            1     100 2025-08-22    Alice Smith      Perth   
3       104            3     300 2025-08-23  Charlie Brown  Melbourne   
4       105            1     175 2025-08-24    Alice Smith      Per

## 4. Grouping and Aggregation

In [9]:
# Create sales data
np.random.seed(42)
sales = pd.DataFrame({
    'date': pd.date_range('2025-08-01', periods=20),
    'store': np.random.choice(['Store_A', 'Store_B', 'Store_C'], 20),
    'product': np.random.choice(['Laptop', 'Phone', 'Tablet'], 20),
    'quantity': np.random.randint(1, 10, 20),
    'revenue': np.random.randint(100, 2000, 20)
})

print("Sales data:")
print(sales.head(10))

# Group by store
store_summary = sales.groupby('store').agg({
    'quantity': 'sum',
    'revenue': ['sum', 'mean', 'count']
})
print("\nStore summary:")
print(store_summary)

# Multiple grouping
product_store = sales.groupby(['product', 'store'])['revenue'].sum().unstack(fill_value=0)
print("\nRevenue by product and store:")
print(product_store)

# Add calculated columns
sales['revenue_per_unit'] = sales['revenue'] / sales['quantity']
sales['day_of_week'] = sales['date'].dt.day_name()

# Group by day of week
daily_pattern = sales.groupby('day_of_week')['revenue'].mean().round(2)
print("\nAverage revenue by day:")
print(daily_pattern)

Sales data:
        date    store product  quantity  revenue
0 2025-08-01  Store_C  Laptop         5     1995
1 2025-08-02  Store_A  Laptop         3     1511
2 2025-08-03  Store_C   Phone         7     1125
3 2025-08-04  Store_C   Phone         5     1121
4 2025-08-05  Store_A  Laptop         9     1513
5 2025-08-06  Store_A  Laptop         7      665
6 2025-08-07  Store_C  Laptop         2     1229
7 2025-08-08  Store_B  Tablet         4     1895
8 2025-08-09  Store_C  Tablet         9     1945
9 2025-08-10  Store_C  Tablet         2     1600

Store summary:
        quantity revenue                   
             sum     sum         mean count
store                                      
Store_A       29    4819   963.800000     5
Store_B       21    6521  1086.833333     6
Store_C       48   11997  1333.000000     9

Revenue by product and store:
store    Store_A  Store_B  Store_C
product                           
Laptop      3689     1986     3224
Phone       1130        0     472

**Exercise 2 — Customer Segmentation (hard)**  
Group customers by total spend into segments: VIP (>500), Regular (200-500), New (<200).


In [79]:
merged['amount'] = np.where(merged['order_id'] == 101, 300,
                            np.where(merged["order_id"] == 102, 150, merged["amount"])
                           )

In [80]:
# Your turn

#Define Condition
total_spend = merged.groupby("customer_id")["amount"].sum()
customer_segment = total_spend.apply(lambda x : "VIP" if x > 500 else "Regular" if 200 <= x <= 500 else "New")
customer_segment = customer_segment.reset_index()
customer_segment.rename(columns={'amount': 'Customer_Segment'}, inplace=True)


merged = pd.merge(merged, customer_segment, on='customer_id', how='left')
merged

,order_id,customer_id,amount,date,name,city,member_since,Customer_Segment
0,101,1,300,2025-08-20,Alice Smith,Perth,2023-01-15,VIP
1,102,2,150,2025-08-21,Bob Jones,Sydney,2023-06-20,New
2,103,1,100,2025-08-22,Alice Smith,Perth,2023-01-15,VIP
3,104,3,300,2025-08-23,Charlie Brown,Melbourne,2024-02-10,Regular
4,105,1,175,2025-08-24,Alice Smith,Perth,2023-01-15,VIP


<details>
<summary><b>Solution</b></summary>

```python
# Calculate customer totals
customer_totals = merged.groupby('name')['amount'].sum().reset_index()
customer_totals.columns = ['customer', 'total_spend']

# Create segments
def segment_customer(spend):
    if spend > 500: return 'VIP'
    elif spend >= 200: return 'Regular'
    else: return 'New'

customer_totals['segment'] = customer_totals['total_spend'].apply(segment_customer)

print("Customer segments:")
print(customer_totals.sort_values('total_spend', ascending=False))

# Segment summary
print("\nSegment distribution:")
print(customer_totals['segment'].value_counts())
```
</details>

## 5. Pivoting and Reshaping

In [11]:
# Create long format data
long_data = pd.DataFrame({
    'date': pd.date_range('2025-08-01', periods=12),
    'metric': ['Sales', 'Costs', 'Profit'] * 4,
    'value': np.random.randint(1000, 5000, 12)
})

print("Long format:")
print(long_data)

# Pivot to wide format
wide_data = long_data.pivot(index='date', columns='metric', values='value')
print("\nWide format (pivoted):")
print(wide_data.head())

# Melt back to long format
melted = wide_data.reset_index().melt(id_vars='date', var_name='metric', value_name='amount')
print("\nMelted back to long:")
print(melted.head())

# Pivot table with aggregation
pivot_table = sales.pivot_table(
    values='revenue',
    index='store',
    columns='product',
    aggfunc='sum',
    fill_value=0,
    margins=True
)
print("\nPivot table with totals:")
print(pivot_table)

Long format:
         date  metric  value
0  2025-08-01   Sales   4342
1  2025-08-02   Costs   1455
2  2025-08-03  Profit   4533
3  2025-08-04   Sales   4798
4  2025-08-05   Costs   2275
..        ...     ...    ...
7  2025-08-08   Costs   3343
8  2025-08-09  Profit   4796
9  2025-08-10   Sales   3767
10 2025-08-11   Costs   4820
11 2025-08-12  Profit   1337

[12 rows x 3 columns]

Wide format (pivoted):
metric       Costs  Profit   Sales
date                              
2025-08-01     NaN     NaN  4342.0
2025-08-02  1455.0     NaN     NaN
2025-08-03     NaN  4533.0     NaN
2025-08-04     NaN     NaN  4798.0
2025-08-05  2275.0     NaN     NaN

Melted back to long:
        date metric  amount
0 2025-08-01  Costs     NaN
1 2025-08-02  Costs  1455.0
2 2025-08-03  Costs     NaN
3 2025-08-04  Costs     NaN
4 2025-08-05  Costs  2275.0

Pivot table with totals:
product  Laptop  Phone  Tablet    All
store                                
Store_A    3689   1130       0   4819
Store_B    1986  

## 6. String Operations and Data Types

In [12]:
# Create messy text data
contacts = pd.DataFrame({
    'name': ['  john smith  ', 'JANE DOE', 'Bob Johnson Jr.', 'alice wong'],
    'email': ['John.Smith@GMAIL.com', 'jane@company.COM', 'bob@email.co', 'Alice@Email.net'],
    'phone': ['0412-345-678', '(04) 9876 5432', '0401234567', '04 1111 2222']
})

print("Messy contact data:")
print(contacts)

# Clean strings
contacts['name_clean'] = contacts['name'].str.strip().str.title()
contacts['email_clean'] = contacts['email'].str.lower()

# Extract domain from email
contacts['domain'] = contacts['email_clean'].str.split('@').str[1]

# Standardize phone numbers
contacts['phone_clean'] = contacts['phone'].str.replace(r'[^0-9]', '', regex=True)

print("\nCleaned contacts:")
print(contacts[['name_clean', 'email_clean', 'phone_clean', 'domain']])

Messy contact data:
              name                 email           phone
0     john smith    John.Smith@GMAIL.com    0412-345-678
1         JANE DOE      jane@company.COM  (04) 9876 5432
2  Bob Johnson Jr.          bob@email.co      0401234567
3       alice wong       Alice@Email.net    04 1111 2222

Cleaned contacts:
        name_clean           email_clean phone_clean       domain
0       John Smith  john.smith@gmail.com  0412345678    gmail.com
1         Jane Doe      jane@company.com  0498765432  company.com
2  Bob Johnson Jr.          bob@email.co  0401234567     email.co
3       Alice Wong       alice@email.net  0411112222    email.net


## 7. Date and Time Operations

In [13]:
# Create time series data
dates = pd.date_range('2025-01-01', periods=100, freq='D')
ts_data = pd.DataFrame({
    'date': dates,
    'sales': np.random.randint(1000, 5000, 100) + np.sin(np.arange(100) * 2 * np.pi / 30) * 500
})

# Extract date components
ts_data['year'] = ts_data['date'].dt.year
ts_data['month'] = ts_data['date'].dt.month
ts_data['day_of_week'] = ts_data['date'].dt.day_name()
ts_data['week'] = ts_data['date'].dt.isocalendar().week

print("Time series with date components:")
print(ts_data.head())

# Resample to weekly
weekly = ts_data.set_index('date')['sales'].resample('W').agg(['mean', 'sum', 'std'])
print("\nWeekly aggregation:")
print(weekly.head())

# Rolling window calculations
ts_data['rolling_mean_7d'] = ts_data['sales'].rolling(window=7).mean()
ts_data['rolling_std_7d'] = ts_data['sales'].rolling(window=7).std()

print("\nWith rolling statistics:")
print(ts_data[['date', 'sales', 'rolling_mean_7d', 'rolling_std_7d']].tail())

Time series with date components:
        date        sales  year  month day_of_week  week
0 2025-01-01  1878.000000  2025      1   Wednesday     1
1 2025-01-02  2179.955845  2025      1    Thursday     1
2 2025-01-03  1994.368322  2025      1      Friday     1
3 2025-01-04  5286.892626  2025      1    Saturday     1
4 2025-01-05  3635.572413  2025      1      Sunday     1

Weekly aggregation:
                   mean           sum          std
date                                              
2025-01-05  2994.957841  14974.789206  1464.270684
2025-01-12  2915.168033  20406.176228  1020.068936
2025-01-19  3410.428571  23873.000000  1661.414047
2025-01-26  2641.689110  18491.823772   751.721868
2025-02-02  2416.076423  16912.534961  1346.988050

With rolling statistics:
         date        sales  rolling_mean_7d  rolling_std_7d
95 2025-04-06  3155.012702      3218.549437     1284.359723
96 2025-04-07  4789.528258      3311.618595     1381.988316
97 2025-04-08  4654.260948      3756.513

**Exercise 3 — Time Series Analysis (hard)**  
Find the best and worst performing days of the week, and calculate week-over-week growth.


In [150]:
avg_sales = ts_data.groupby('day_of_week')['sales'].mean().reset_index()
worst_day = avg_sales[avg_sales['sales'] == avg_sales['sales'].min()]
best_day =  avg_sales[avg_sales['sales'] == avg_sales['sales'].max()]
print("\nWorst day is:")
print(worst_day)
print("\nBest day is:")
print(best_day)
weekly_sales = ts_data.groupby('week')['sales'].sum().reset_index()
test['sales'].pct_change() * 100


Worst day is:
  day_of_week        sales
0      Friday  2670.760938

Best day is:
  day_of_week        sales
4    Thursday  3535.875155


0           NaN
1     36.270207
2     16.989091
3    -22.540846
4     -8.540471
        ...    
10    18.692099
11   -28.403543
12    18.741359
13     8.997257
14   -25.163366
Name: sales, Length: 15, dtype: float64

## 8. Mini-Challenges
- **M1 (easy):** Create a function to detect duplicate rows based on subset of columns
- **M2 (medium):** Implement a data validation function that checks data types and ranges
- **M3 (hard):** Create a pipeline that cleans, transforms, and aggregates raw sales data

In [15]:
# Your turn - try the challenges!


<details>
<summary><b>Solutions</b></summary>

```python
# M1 - Duplicate detection
def find_duplicates(df, subset=None):
    duplicates = df[df.duplicated(subset=subset, keep=False)]
    return duplicates.sort_values(subset if subset else df.columns.tolist())

# M2 - Data validation
def validate_data(df, rules):
    """
    rules = {
        'column_name': {'type': str, 'min': 0, 'max': 100, 'required': True}
    }
    """
    issues = []
    for col, rule in rules.items():
        if col not in df.columns and rule.get('required'):
            issues.append(f"Missing required column: {col}")
            continue
        
        if 'type' in rule:
            wrong_type = df[col].apply(lambda x: not isinstance(x, rule['type']))
            if wrong_type.any():
                issues.append(f"{col}: {wrong_type.sum()} type mismatches")
        
        if 'min' in rule:
            below_min = df[col] < rule['min']
            if below_min.any():
                issues.append(f"{col}: {below_min.sum()} values below {rule['min']}")
    
    return issues

# M3 - Data pipeline
def sales_pipeline(raw_df):
    # Clean
    df = raw_df.copy()
    df = df.dropna(subset=['product', 'revenue'])
    df['date'] = pd.to_datetime(df['date'])
    
    # Transform
    df['month'] = df['date'].dt.to_period('M')
    df['revenue_category'] = pd.cut(df['revenue'], 
                                     bins=[0, 500, 1000, float('inf')],
                                     labels=['Low', 'Medium', 'High'])
    
    # Aggregate
    summary = df.groupby(['month', 'product']).agg({
        'revenue': ['sum', 'mean', 'count'],
        'quantity': 'sum'
    }).round(2)
    
    return summary

# Test pipeline
result = sales_pipeline(sales)
print("Pipeline output:")
print(result.head())
```
</details>

## Wrap-Up
✅ You can read and clean messy real-world data  
✅ You mastered merging, grouping, and pivoting  
✅ You can handle dates, strings, and missing values  

**Next:** EDA - Exploring and understanding your cleaned data!
